In [3]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model

Using TensorFlow backend.


In [0]:
# Package for face landmark detection
# !pip install mlxtend
import mlxtend

In [5]:
import mlxtend
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

# New Section

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [7]:
! ls '/content/drive/My Drive/CSCE_689'


 0.avi	  40.avi				     Labels_final.csv
 10.avi   41.avi				     model1.h5
 11.avi   42.avi				     model.h5
 12.avi   43.avi				     model.json
 13.avi   44.avi				     model.png
 14.avi   45.avi				     optimization
 15.avi   46.avi				     Prediction
 16.avi   47.avi				     README.md
 17.avi   48.avi				     Test
 18.avi   49.avi				    'Test10 - NN.png'
 19.avi   4.avi					    'Test10 - TL.png'
 1.avi	  50.avi				     Test_1.avi
 20.avi   51.avi				    'Test1 - NN.png'
 21.avi   52.avi				     Test_2.avi
 22.avi   53.avi				    'Test2 - NN.png'
 23.avi   54.avi				     Test_3.avi
 24.avi   55.avi				    'Test3 - NN.png'
 25.avi   56.avi				     Test_4.avi
 26.avi   57.avi				    'Test4 - NN.png'
 27.avi   58.avi				     Test_5.avi
 28.avi   5.avi					    'Test5 - NN.png'
 29.avi   6.avi					     Test_video
 2.avi	  7.avi					    'timeLabel_test10 NN.json'
 30.avi   8.avi					    'timeLabel_test10 TL.json'
 31.avi   9.avi					    'timeLabel_test1 NN.json'
 32.avi   best_model				 

In [8]:
cd '/content/drive/My Drive/CSCE_689/'

/content/drive/My Drive/CSCE_689


### Define and compute inputs terms 

In [0]:
def eye_aspect_ratio(eye):
	
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [0]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [0]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [0]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [0]:
def jaw_dist(jaw,nose):
  list= np.array([distance.euclidean(d, nose) for d in jaw])
  return list.max()/list.mean()



In [0]:
def getFrame(sec):
    start = 0
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [0]:
def get_landmarks(vidcap,i):
  sec = 0
  frameRate = 0.5
  success, image = getFrame(sec)
  count = 0
  data=[]
  labels=[]
  timestamp=[]
  while success and count < int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
      landmarks = extract_face_landmarks(image)
      try:
          sum(sum(landmarks))
          if sum(sum(landmarks)) != 0:
              count += 1
              data.append(landmarks)
              labels.append([i])
              # cv2.imwrite("frame%d.jpg" % count, image)
              timestamp.append(sec)
              sec = sec + frameRate
              sec = round(sec, 3)
              success, image = getFrame(sec)

              # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          else:
              sec = sec + frameRate
              sec = round(sec, 3)
              success, image = getFrame(sec)
              print(str(i)+' not detected')
      except:
          # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          count=int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
          pass
      # print(count)
  return data,labels,timestamp

In [0]:
# Generate Landmarks for all 41 training videos
from scipy.spatial import distance
import cv2
data = []
labels = []
frmes=[]
timestamp=[]
for i in range(59):
    vidcap = cv2.VideoCapture(str(i) + '.avi')
    data.extend(get_landmarks(vidcap,i)[0])
    labels.extend(get_landmarks(vidcap,i)[1])
    timestamp.extend(get_landmarks(vidcap,i)[2])
   
        

In [0]:
##Create Features
import numpy as np
import math
data = np.array(data)
features = []
main_list=[]
jaw_list=[]
for d in data:
    eye = d[36:68]
    nose=d[28]
    jaw=d[5:13]
    main_list.append(eye)
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    cir = circularity(eye)
    movt = jaw_dist(jaw,nose)
    mouth_eye = mouth_over_eye(eye)
    features.append([ear, mar, cir, mouth_eye,movt])

In [0]:
import pandas as pd
features = np.array(features)
features.shape
time= np.array(timestamp)
x=pd.DataFrame()
x['label']=labels
x['timestamp']=time

In [0]:
# Save files
np.save(open('Data2.npy', 'wb'),data) ## Save landmark data
np.savetxt('Timestamps.csv',time)
np.savetxt("Features_final.csv", features, delimiter = ",")
np.savetxt("Labels.csv", labels, delimiter = ",")

In [0]:
## Save a combine file of label and timestamp. It will be converted to Labels_final.csv which will have labels of 0 (for not yawning) and 1 (for yawning)
x.to_csv("combinedlabels.csv") 

In [0]:
# Loading the files 
import numpy as np
import pandas as pd
labels=pd.read_csv('Labels_final.csv',header=None)
features=pd.read_csv('Features_final.csv',header=None)

In [17]:
## Generating train images for transfer learning
import cv2
j=0
c_a=0
c_b=0
for i in range(52):
  vidcap = cv2.VideoCapture(str(i) + '.avi')
  print(i)
  sec = 0
  frameRate = 0.5
  success, image = getFrame(sec)
  count = 0
  while success and count <= int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
      landmarks = extract_face_landmarks(image)
      try:
          sum(sum(landmarks))
          if sum(sum(landmarks)) != 0:
            count += 1
            
            if labels.iloc[j,2]==0:
              # print('0')
              c_a=c_a+1
              cv2.imwrite('/content/drive/My Drive/CSCE_689/Train/Class_0/'+format(c_a)+str(".jpg"), image)
            else:
              # print('1')
              c_b=c_b+1
              cv2.imwrite('/content/drive/My Drive/CSCE_689/Train/Class_1/'+format(c_b)+str(".jpg"), image)
            sec = sec + frameRate
            sec = round(sec, 3)
            success, image = getFrame(sec)
            j=j+1
            # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          else:
            sec = sec + frameRate
            sec = round(sec, 3)
            success, image = getFrame(sec)
            # print(str(i)+' not detected')
      except:
          # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          count=int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
          pass
      # print(count)

0


/usr/local/lib/python3.6/dist-packages/mlxtend/image/extract_face_landmarks.py:61: UserWarning: No face detected.
  warnings.warn('No face detected.')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [0]:
## Generating validation images for transfer learning
import cv2
j=4014
c_a=0
c_b=0
for i in range(52,59):
  vidcap = cv2.VideoCapture(str(i) + '.avi')
  sec = 0
  frameRate = 0.5
  success, image = getFrame(sec)
  count = 0
  while success and count < int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
      landmarks = extract_face_landmarks(image)
      try:
          sum(sum(landmarks))
          if sum(sum(landmarks)) != 0:
            count += 1
            j=j+1
            if labels.iloc[j,2]==0:
              # print('0')
              c_a=c_a+1
              cv2.imwrite('/content/drive/My Drive/CSCE_689/Test/Class_0/'+format(c_a)+str(".jpg"), image)
            else:
              # print('1')
              c_b=c_b+1
              cv2.imwrite('/content/drive/My Drive/CSCE_689/Test/Class_1/'+format(c_b)+str(".jpg"), image)
            sec = sec + frameRate
            sec = round(sec, 3)
            success, image = getFrame(sec)
            
            # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          else:
            sec = sec + frameRate
            sec = round(sec, 3)
            success, image = getFrame(sec)
            # print(str(i)+' not detected')
      except:
          # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          count=int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
          pass
      # print(count)

/usr/local/lib/python3.6/dist-packages/mlxtend/image/extract_face_landmarks.py:61: UserWarning: No face detected.
  warnings.warn('No face detected.')


In [0]:
j=4014
import cv2
for i in range(52,59):
  vidcap = cv2.VideoCapture(str(i) + '.avi')
  sec = 0
  frameRate = 0.5
  success, image = getFrame(sec)
  count = 0
  while success and count < int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
      landmarks = extract_face_landmarks(image)
      try:
          sum(sum(landmarks))
          if sum(sum(landmarks)) != 0:
            count += 1
            j=j+1
            cv2.imwrite('/content/drive/My Drive/CSCE_689/Prediction/'+format(j)+str(".jpg"), image)
            sec = sec + frameRate
            sec = round(sec, 3)
            success, image = getFrame(sec)
            
            # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          else:
            sec = sec + frameRate
            sec = round(sec, 3)
            success, image = getFrame(sec)
            # print(str(i)+' not detected')
      except:
          # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          count=int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
          pass

/usr/local/lib/python3.6/dist-packages/mlxtend/image/extract_face_landmarks.py:66: UserWarning: No face detected.
  warnings.warn('No face detected.')
